In [1]:
import wmi
import gc
from win32api import GetFileVersionInfo, HIWORD, LOWORD
wmic = wmi.WMI()
process = wmic.Win32_Process(name="cloudmusic.exe")
process = [p for p in process if '--type=' not in p.ole_object.CommandLine]
if not process:
    raise RuntimeError('No candidate process found')
elif len(process) != 1:
    raise RuntimeError('Multiple candidate processes found!')
else:
    process = process[0]
    pid = process.ole_object.ProcessId
    ver_info = GetFileVersionInfo(process.ole_object.ExecutablePath, '\\')
    ver = f"{HIWORD(ver_info['FileVersionMS'])}.{LOWORD(ver_info['FileVersionMS'])}." \
          f"{HIWORD(ver_info['FileVersionLS'])}.{LOWORD(ver_info['FileVersionLS'])}"
    print(f'found version: {ver}')
    del process
    del wmic
    gc.collect()

found version: 3.0.1.5106


In [7]:
from pyMeow import open_process, get_module, r_float64, r_bytes, r_uint

process = open_process(pid)
base_address = get_module(process, 'cloudmusic.dll')['base']
print('base_address: 0x%X' % base_address)

base_address: 0x7FF822920000


In [9]:
type(r_float64(process, base_address + 0x18ed7c8))

float

In [16]:
import math
from tqdm.auto import tqdm

found_songarray = False
found_length = False
found_current = False
for offset in tqdm(range(0x18e0000, 0x343b000)): # Change to range of cloudmusic.dll in cheat engine
    if found_songarray and found_length and found_current:
        break
    if not found_songarray:
        try:
            songid_array = r_uint(process, base_address + offset)
            string = '1458767558'  # Change to current song ID
            song_id = r_bytes(process, songid_array, len(string) * 2).decode('utf-16')
            if song_id == string:
                print('found song array offset at 0x%X: %s' % (offset, song_id))
                found_songarray = True
        except Exception as e:
            pass
    
    if not found_length:
        try:
            length = r_float64(process, base_address + offset)
            if math.floor(length) == 255:  # Change to song length, in seconds
                print('found length at 0x%X: %f' % (offset, length))
                found_length = True
        except Exception as e:
            pass

    if not found_current:
        try:
            current = r_float64(process, base_address + offset)
            if math.floor(current) == 33:  # Change to current progress, in seconds
                print('found current at 0x%X: %f' % (offset, current))
                found_current = True
        except Exception as e:
            pass

  0%|          | 0/28684288 [00:00<?, ?it/s]

found current at 0x18ED7C8: 33.710000
found length at 0x192D548: 255.240000
